### 앙상블 (Ensemble)
- 여러개의 분류 모델을 조합해서 더 나은 성능을 내는 방법
- Decision Tree 모델을 증가시켜 나온 RandomForest가 대표적임

#### RandomForest
- 부트스트랩 샘플을 사용합니다. 부트스트랩 샘플링은 중복을 허용하는 샘플링 방법
- 샘플링 후에 샘플을 다시 복구하고 다시 샘플링하는 방법입니다.
- 이와 같이 진행하는 이유는 결정트리에서 과대적합을 방지할 수 있기 때문이다.
- 각 결정트리에서 나오는 확률의 합을 트리 개수로 나누어 결정하는 모델
- 특성의 개수를 제곱근으로 합니다.

In [1]:
import pandas as pd

wine = pd.read_csv('../Data/wine.csv')


In [2]:
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [6]:
wine[['alcohol','sugar','pH']]

,alcohol,sugar,pH
0,9.4,1.9,3.51
1,9.8,2.6,3.20
2,9.8,2.3,3.26
3,9.8,1.9,3.16
4,9.4,1.9,3.51
...,...,...,...
6492,11.2,1.6,3.27
6493,9.6,8.0,3.15
6494,9.4,1.2,2.99
6495,12.8,1.1,3.34


In [4]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
data

array([[ 9.4 ,  1.9 ,  3.51],
       [ 9.8 ,  2.6 ,  3.2 ],
       [ 9.8 ,  2.3 ,  3.26],
       ...,
       [ 9.4 ,  1.2 ,  2.99],
       [12.8 ,  1.1 ,  3.34],
       [11.8 ,  0.8 ,  3.26]])

In [20]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,target,test_size=0.2,random_state=42)

In [21]:
# RandomForest
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [22]:
# 주요 Feature 확인
rf.fit(train_input,train_target)
print(rf.feature_importances_)


[0.23167441 0.50039841 0.26792718]


In [23]:
# 부트스트랩 결정시 남는 샘플 (oob: out of back)도 특성으로 구분할 수 있다.
rf = RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
rf.oob_score_

0.8934000384837406

> oob를 사용해도 90%의 예측이 나온다    
> 따로 검증셋을 구성하지 않아도 oob로 검증세트의 역할을 대신할 수 있다.

> RandomForest는 기본값만으로도 높은 성능을 발휘하므로 자주 사용되는 모델 (알고리즘) 중 하나이다.

---
### Extra Tree
- 기본적으로 100개의 트리를 사용
- 노드 분할시 특성의 제곱근을 개수로 사용
- 특성의 선택을 랜덤하게 선택한다.
- 특성의 선택을 랜덤하게 하므로 속도는 랜덤포레스트보다 빠르다

In [24]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(et, train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [25]:
et.fit(train_input,train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

---
### Gradient Boosting (그래디언트 부스팅)
- 가장 유명한 알고리즘 중 하나이다.
- 경사하강법처럼 손실함수를 사용
- 손실함수를 보고 트리를 추가하여 최적의 값 도출하는 방법
- 경사를 이동하면서 경사의 이동거리를 제어하는 learning-rate(defalut:0.1) 를 사용한다.
- max_depth를 3으로 제어하여 깊이가 낮으므로 과대적합 방지
- 단점은 손실함수를 보고 트리를 추가하면서 진행하는 모델이므로 병렬처리를 할 수 없다.

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


tree 추가를 500개까지 하고 learning rate를 0.2로 변경

In [27]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [28]:
gb.fit(train_input,train_target)
gb.feature_importances_

array([0.15872278, 0.68010884, 0.16116839])

---
### 히스토그램 기반 그래디언트 부스팅 (Histogram gradient boosting)
- 훈련 데이터를 256개 구간으로 나누어서 훈련시키는 방법.
- 특성의 범위가 제한되어있어 빠른 속도를 제공한다.
- 제한된 구간이므로 과대적합을 방지한다.
- 아직은 실험단계인 모델이다.

In [29]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


# XGBoost
- kaggle에서 많이 사용

In [35]:
# !pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [31]:
from xgboost import XGBClassifier

In [32]:
xgb = XGBClassifier(
    tree_method = 'hist',
    random_state = 42,
    use_label_encoder = False,
    eval_metrics = 'logloss'
)

/Users/ethan/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


---
### 번외 기능
#### Permutation Importance (치환 중요도)
- 각 Feature별 Sample을 섞어서 계산을 한 후에 원래 Sample들과의 차이를 계산해서 차이가 많이 나는 Feature가 중요하다고 판단.
- 즉 어느 Feature가 중요한지 파악하는 방법
- 어떤 모델에도 사용가능하며 특성을 파악하는 주요 기준으로 사용된다. <- (*권장사항*)

In [33]:
from sklearn.inspection import permutation_importance

xgb.fit(train_input, train_target)
result = permutation_importance(xgb,train_input,train_target,n_repeats=10,random_state=42,n_jobs=-1)
result.importances_mean

[17:59:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "eval_metrics" } are not used.



array([0.11306523, 0.25508947, 0.10696556])

> vs gb

In [34]:
gb.fit(train_input, train_target)
result = permutation_importance(gb,train_input,train_target,n_repeats=10,random_state=42,n_jobs=-1)
result.importances_mean

array([0.09661343, 0.24265923, 0.09120647])

---
# 앙상블 정리   
- 앙상블 학습은 정형데이터에서 가장 뛰어난 성능을 내는 머신러닝 알고리즘 중 하나이다.   

#### 랜덤포레스트   
: 부트스트랩 샘플 사용, 대표 앙상블 학습 알고리즘   

#### 엑스트라트리   
: 결정트리의 노드를 랜덤하게 분할함.    

#### 그래디언트 부스팅  
: 트리의 손실을 보완하는 식으로 얕은 결정트리를 연속하여 추가함.    

#### 히스토그램 기반 그래디언트 부스팅  
: 훈련 데이터를 256개 정수 구간으로 나누어 빠르고 높은 성능을 냄.   

#### 기타
- XGBoost